In [2]:
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings.ollama import OllamaEmbeddings

c:\Users\QIWU\AppData\Local\anaconda3\envs\science\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
chroma_path = "../chroma"

In [4]:
vectordb = Chroma(persist_directory=chroma_path, embedding_function = OllamaEmbeddings(model='nomic-embed-text'))

In [5]:
retriever = vectordb.as_retriever()

In [6]:
URI = "https://repro-toxicity-xrbuc.eastus2.inference.ml.azure.com/score"
KEY = "9aIh4YiafyykA1e9okBCiwz6gupwDskX"

In [7]:
from langchain_community.chat_models.azureml_endpoint import AzureMLChatOnlineEndpoint, CustomOpenAIChatContentFormatter
from langchain_community.llms.azureml_endpoint import AzureMLEndpointApiType

llm = AzureMLChatOnlineEndpoint(
    endpoint_url=URI,
    endpoint_api_type=AzureMLEndpointApiType.dedicated,
    endpoint_api_key=KEY,
    content_formatter=CustomOpenAIChatContentFormatter(),
    model_kwargs={"max_tokens": 512},
)


In [29]:
llm.invoke("Hello, how are you?")

BaseMessage(content="I'm just a language model, I don't have emotions or feelings like humans do, but I'm functioning properly and ready to help you with any questions or tasks you have. How about you? How's your day going so far?", type='assistant', id='run-e77330f5-5a5f-4734-9bab-c4067cff9d52-0')

In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context. 
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(llm, question_answering_prompt)

In [9]:
from langchain_core.messages import AIMessage, HumanMessage

query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
        ),
    ]
)

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1,
        # If only one message, then we just pass that message's content to retriever
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
    query_transform_prompt | llm | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

In [11]:
SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context. 
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(llm, question_answering_prompt)

In [12]:
question = "What happened with  OECD  Working  Group in  Copenhagen  in  June  1995?"

In [15]:
from langchain_core.runnables import RunnablePassthrough
conversational_retrieval_chain = RunnablePassthrough.assign(
            context=query_transforming_retriever_chain,
        ).assign(
            answer=document_chain,
        )

In [32]:
response = conversational_retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="You are a chat bot that answers questions about test guidelines. "),
            AIMessage(content="Hello, I'm a chatbot built to help you query OECD test guidelines."),
            HumanMessage(content=question),
        ]
    }
)

In [33]:
response

{'messages': [HumanMessage(content='What happened with  OECD  Working  Group in  Copenhagen  in  June  1995?')],
 'context': [Document(page_content='In Copenhagen in June 1995, an OECD Working Group on Reproduction and Developmental', metadata={'id': '..\\documents\\OECD TG 416 (2001).pdf:0:2', 'page': 0, 'source': '..\\documents\\OECD TG 416 (2001).pdf'}),
  Document(page_content='OECD/OCDE             441 \n \n5 © OCDE, (2009)  \n Housing and feeding conditions', metadata={'id': '..\\documents\\OECD TG 441 (2009).pdf:4:0', 'page': 4, 'source': '..\\documents\\OECD TG 441 (2009).pdf'}),
  Document(page_content='OECD/OCDE             441 \n \n17 © OCDE, (2009)  \n ANNEX 2  \nNote : Document prepared by the Secretariat of the Test Guidelines Prog ramme based on the agreement reached at the 6th Meeting of the E DTA Task Force\nOECD Conceptual Framework for the Testing and Assessment of Endocrine Disrupting Chemicals\nLevel 1\nSorting & prioritization based \nupon existing information\nLe

In [28]:
response['answer']

'According to the provided context, an OECD Working Group on Reproduction and Developmental Toxicology (RTD) was held in Copenhagen in June 1995.'